## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [2]:
path__tweets_csv = '/home/stelios/Desktop/Honours Project/Samples/tweet_data/tweets_joh_all.csv'
path_users_csv = '/home/stelios/Desktop/Honours Project/Samples/user_data/joh.csv'

joh_join_df = pd.read_csv(path__tweets_csv,index_col=None, header=0, low_memory = False).applymap(str).join(pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str).set_index('user_id'), on='user_id')

## Extract, Count & Sort Emojis for all

In [3]:
extract = Extractor()

count = extract.count_all_emoji(joh_join_df['text'])

# Create panda dataframe to store emojis with counts
emojis_count_df = pd.DataFrame(columns = ['Emoji','Count'])
for emoji in count:
    emojis_count_df = emojis_count_df.append({'Emoji' : emoji , 'Count' : count[emoji]} , ignore_index=True)

# Sort dataframe by count
emojis_count_df = emojis_count_df.sort_values(by=['Count'])
emojis_count_df.tail(20)

Emoji   Count
0       👀   14580
24      💕   15555
189     😹   16310
10      😁   17479
22      🤔   18054
110     🙌   18165
106     👏   19161
169     🙏   19699
108     😢   25902
5       😊   26942
15      😅   29872
32      😩   33553
117     💀   35877
38      💔   40228
17      ❤   92624
23      😍   93776
33      🔥  136057
44      🤣  189896
25      😭  190321
1       😂  927229

## TABLE

In [4]:
# Load users
joh_users_df = pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str)

# Calculate lengths
length_emojis = len(emojis_count_df['Emoji'].values)
print(length_emojis)
length_users = len(joh_users_df['user_id'])

# Create zeros matrix
zeros_matrix= numpy.zeros(shape=(length_users,length_emojis))

# Fill pandas dataframe with zeros
emojis_count_create_df = pd.DataFrame(zeros_matrix,columns = emojis_count_df['Emoji'].values)
emojis_count_create_df

# Initialise extractor
extract = Extractor()

# Add the frequencies of emojis each emoji by User
for enum,user in enumerate (joh_users_df['user_id']):
    user_posts= "".join(joh_join_df.loc[joh_join_df['user_id'] == user, 'text'])
    text_count_emoji = extract.count_emoji(user_posts, check_first=True)
    if(len(text_count_emoji)==0):
        pass
    else:
        for emoji in text_count_emoji:
            emojis_count_create_df.at[enum, emoji] = text_count_emoji[emoji]

# Describe
emojis_count_create_df.info(memory_usage='deep')

2149
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 2183 entries, 🧖‍♂️ to 🇧🇫
dtypes: float64(2183)
memory usage: 166.5 MB


In [5]:
# Fill n/a with 0.0
emojis_count_create_df = emojis_count_create_df.fillna(0.0)

# Store table
emojis_count_create_df.to_csv("/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/joh_per_user_emoji_counts.csv",encoding='utf-8', index=False)

In [6]:
# Testing
emojis_count_create_df['😂']

0          0.0
1          0.0
2        148.0
3          4.0
4          0.0
5         44.0
6          0.0
7          0.0
8       1445.0
9          0.0
10         0.0
11        11.0
12         0.0
13        97.0
14       143.0
15         1.0
16       220.0
17        11.0
18        26.0
19        26.0
20         0.0
21        22.0
22        15.0
23         0.0
24         0.0
25        44.0
26       284.0
27         0.0
28        66.0
29        55.0
         ...  
9970      98.0
9971      10.0
9972       0.0
9973      31.0
9974       0.0
9975     238.0
9976      43.0
9977      99.0
9978       0.0
9979       0.0
9980       0.0
9981       0.0
9982      28.0
9983      14.0
9984       0.0
9985     150.0
9986       0.0
9987     150.0
9988       1.0
9989       0.0
9990       0.0
9991       0.0
9992       0.0
9993     164.0
9994      10.0
9995       0.0
9996      83.0
9997      22.0
9998       7.0
9999      61.0
Name: 😂, Length: 10000, dtype: float64